# Simulating Geometric Brownian Motion (GBM) in Python

Geometric Brownian motion (GBM) S is defined by S0 > 0 and the dynamics as defined in the following Stochastic Differential Equation (SDE):


$\Large dS_t = \mu S_t dt + \sigma S_t dW_t$

Integrated Form:

 - $\log S_t = \log S_0 + \int_{0}^{t} (\mu-\frac{\sigma^2}{2}) \,ds + \int_{0}^{t} \sigma \,dW_s $

 - $\log S_t = \log S_0 + (\mu-\frac{\sigma^2}{2})t + \sigma W_t $

 - $\log S_t \sim N(\log S_0 + (\mu-\frac{\sigma^2}{2})t, \sigma^2 t)$

Explicit Expression:

$\Large S_t = S_0 {\rm e}^{(\mu-\frac{\sigma^2}{2})t + \sigma W_t}$

### Parameters

### Simulating GBM Paths

We have two options here, we can either:
- (like here) simulate the stock price directly throughout the simulation and multiple the exponential terms together at each time step; or
- we can simulate the log normal distribution and cumulatively add the terms along each sample path


$\Large S_t = S_0 {\rm e}^{(\mu-\frac{\sigma^2}{2})t + \sigma W_t}$

In [1]:
import numpy as np
import random
import warnings
import pandas as pd
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [12]:
database_1 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 1.csv')
database_2 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 2.csv')
database_3 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 3.csv')
database_4 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 4.csv')
database_5 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 5.csv')
database_6 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 6.csv')
database_7 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 7.csv')
database_8 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 8.csv')
database_9 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 9.csv')
database_10 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 10.csv')
database_11 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 11.csv')
database_12 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 12.csv')
database_13 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 13.csv')
database_14 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 14.csv')
database_15 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 15.csv')
database_16 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 16.csv')
database_17 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 17.csv')
database_18 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 18.csv')
database_19 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 19.csv')
database_20 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 20.csv')
database_21 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 21.csv')
database_22 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 22.csv')
database_23 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 23.csv')
database_24 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 24.csv')
database_25 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 25.csv')
database_26 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 26.csv')
database_27 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 27.csv')
database_28 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 28.csv')
database_29 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 29.csv')
database_30 = pd.read_csv('c:\\Users\\Infovesta PC-57\\quanti-trading-plan\\data\\database_daily_update\\Database Part 30.csv')
database = pd.concat([database_1,database_2,database_3,database_4,database_5,database_6,database_7,database_8,database_9,database_10,
                      database_11,database_12,database_13,database_14,database_15,database_16,database_17,database_18,database_19,database_20,
                      database_21,database_22,database_23,database_24,database_25,database_26,database_27,database_28,database_29,database_30])
database = database.drop(columns='Unnamed: 0')
#Substitusi nilai Open, High, Low Price yg 0 dengan Close Price
database['Open Price'] = np.where(database['Open Price'].eq(0),database['Close Price'],database['Open Price'])
database['High Price'] = np.where(database['High Price'].eq(0),database['Close Price'],database['High Price'])
database['Low Price'] = np.where(database['Low Price'].eq(0),database['Close Price'],database['Low Price'])
df = database[['Kode','Date','Close Price']]


In [13]:
def data_preprocessing(data,stock, date, period):
    data = data[data['Kode']==stock]
    data = data.sort_values(by='Date')
    data_selected = data[data['Date']<=date]
    data_selected = data_selected.tail(period)
    prices = data_selected['Close Price'].values
    logprices = np.log(prices)
    logreturns = logprices[1:] - logprices[:-1]
    mu = np.mean(logreturns)
    sigma = np.std(logreturns)
    return data,data_selected, mu, sigma
            
def simulate_1d_gbm(nsteps, mu, sigma, ref_price):
    steps = [ (mu - (sigma**2)/2) + np.random.randn()*sigma for i in range(nsteps-1) ]
    steps.insert(0,0)
    res = ref_price*np.exp(np.cumsum(steps))
    return res

In [32]:
param_period = 252 #param_period adalah periode pengambilan mu dan sigma, pred_period adalah holding period indeks. dibedakan karena mengurangi efek seasonality. contoh: jika param_period = pred_period = 63 maka seasonality terjadi karena momentum sering berbalik arah tiap 3 bulan
pred_period = 63
stock = 'ELSA'
sim_num = 100000
date = '2023-01-04'

data,data_selected,mu, sigma = data_preprocessing(data=df,stock=stock, date=date, period=param_period)
ref_price = data[data['Date']==date]['Close Price'].iloc[0]
y_list = []
for i in list(range(0,sim_num)):
    y = simulate_1d_gbm(nsteps=param_period, mu=mu, sigma=sigma, ref_price=ref_price)
    y_list.append(y[pred_period])   #Ubah prediction period disini

exp_price = np.mean(y_list)
date_list = list(data['Date'])
date_i = date_list.index(date) + pred_period
date_ = date_list[date_i]
act_price = data[data['Date']==date_]['Close Price'].iloc[0]
print(f'Mean is {mu}, stdev is {sigma}')
print(f'Ref price: {ref_price}')
print(f'Expected end price: {exp_price}')
print(f'Actual end price: {act_price}')
for i in [1,2,5,10]:
    threshold = np.percentile(y_list, i)
    print(f'Bottom price at {i}% probability: {threshold}')


Mean is 0.0004027733739895166, stdev is 0.024860706056819223
Ref price: 312.0
Expected end price: 320.01227445484614
Actual end price: 308.0
Bottom price at 1% probability: 198.49373739067215
Bottom price at 2% probability: 209.71505145868989
Bottom price at 5% probability: 227.27616339180744
Bottom price at 10% probability: 243.98658262665106


In [ ]:
list_ = [120, 20, 30, 40, 50, 60, 70, 80, 90, 100]
lower_threshold = np.percentile(list_, 5)
upper_threshold = np.percentile(list_, 95)

#bottom_5_percent = [x for x in data if x <= lower_threshold]
#top_5_percent = [x for x in data if x >= upper_threshold]

#print("Bottom 5% values:", bottom_5_percent)
#print("Top 5% values:", top_5_percent)


In [25]:
lower_threshold, upper_threshold

(np.float64(24.5), np.float64(110.99999999999997))

In [ ]:
df 